In [ ]:
# follows: https://gist.github.com/GeorgeDittmar/5c57a35332b2b5818e51618af7953351  and https://github.com/huggingface/transformers/blob/master/examples/tensorflow/language-modeling/run_clm.py


import tensorflow as tf
import os
from functools import partial
import numpy as np

In [ ]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 80215, done.
remote: Total 80215 (delta 0), reused 0 (delta 0), pack-reused 80215
Receiving objects: 100% (80215/80215), 62.72 MiB | 15.89 MiB/s, done.
Resolving deltas: 100% (57429/57429), done.


In [ ]:
os.chdir("./transformers")

In [ ]:
#using the transformers library by huggingface
!pip install .

from transformers import GPT2Tokenizer
from transformers import AdamWeightDecay
from transformers import TFGPT2LMHeadModel

Processing /content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 43 kB 1.0 MB/s 
     |████████████████████████████████| 636 kB 7.2 MB/s 
     |████████████████████████████████| 895 kB 42.2 MB/s 
     |████████████████████████████████| 3.3 MB 63.5 MB/s 
  Created wheel for transformers: filename=transformers-4.10.0.dev0-py3-none-any.whl size=2644547 sha256=9b78a7913c1476bb72f71c1dfcf7309f7fbcccd69f2ce54f93676dec4d037864
  Stored in directory: /tmp/pip-ephem-wheel-ca

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
from datasets import load_dataset

     |████████████████████████████████| 264 kB 5.1 MB/s 
     |████████████████████████████████| 118 kB 87.0 MB/s 
     |████████████████████████████████| 243 kB 82.0 MB/s 


In [ ]:
data_files = {}
data_files["train"] = "/content/drive/My Drive/Datasets/all_genres/kaggle_train.csv"
data_files["validation"] = "/content/drive/My Drive/Datasets/all_genres/kaggle_validation.csv"
#huggingface library call
raw_datasets = load_dataset('csv', data_files=data_files)

Using custom data configuration default-a4457ccd50a8a61b


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a4457ccd50a8a61b/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'lyrics'],
        num_rows: 225279
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'lyrics'],
        num_rows: 11857
    })
})

In [ ]:
column_names = raw_datasets["train"].column_names

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", eos_token = "<|endoftext|>")

In [ ]:
#we add <|endoftext|> token to each song while tokenizing
def tokenize_function(examples):
        songs = [song + "<|endoftext|>" for song in examples["lyrics"]]
        tokens = tokenizer(songs, truncation=True , max_length =1024) 
        return tokens

In [ ]:
#for each song the tokenizer returns an attention mask and the input ids
tokenized_datasets = raw_datasets.map(
        tokenize_function,
        batched=True,
        remove_columns=column_names,
        load_from_cache_file=0,
        desc="Running tokenizer on dataset",
    )

Running tokenizer on dataset:   0%|          | 0/226 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 225279
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 11857
    })
})

In [ ]:
block_size = tokenizer.model_max_length

In [ ]:
def group_texts(examples):
        # Concatenate all texts.
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # as gpt2 does not use a padding token we will have to drop parts of the last song
        if total_length >= block_size:
            total_length = (total_length // block_size) * block_size
        # Split by chunks of max_len.
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
        #we need to add the labels because tensorflow demands them eventhough gpt2 does only use input_ids
        result["labels"] = result["input_ids"].copy()
        return result
        

In [ ]:
# grouping the tokenized dataset into blocks of 1024. 
# gpt-2 was trained on text where seperation was done by the <|endoftext|> token rather than seperation by text 
# this means within one block we can have multiple songs
# e.g. Block1 = song1<|endoftext|>song2, Block2 = song2(the rest)<|endoftext|>song3<|endoftext|>

lm_datasets = tokenized_datasets.map(
        group_texts,
        batched=True,
        load_from_cache_file=1,
        desc=f"Grouping texts in chunks of {block_size}",
    )

Grouping texts in chunks of 1024:   0%|          | 0/226 [00:00<?, ?ba/s]

Grouping texts in chunks of 1024:   0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
# as you can see the num_rows has decreased substancially as most songs are shorter than 1024 tokens
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 66881
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 3526
    })
})

In [ ]:
train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]

In [ ]:
eval_dataset[1]["input_ids"]

[1101, 319, 616, 835, 1312, 1101, 319, 616, 835, 287]

In [ ]:
model = TFGPT2LMHeadModel.from_pretrained("/content/drive/My Drive/NLP/GPT-2/Model") # path to our pretrained model, in case of fresh training it can be replaced with e.g. "gpt2-medium"

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/My Drive/GPT-2/Model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
#vocab_size of gpt-2
len(tokenizer)

50257

In [ ]:
#ensuring the right embedding size, should however not be necessary as the standard gpt-2 tokenizer is used
model.resize_token_embeddings(len(tokenizer))

<tf.Variable 'tfgp_t2lm_head_model/transformer/wte/weight:0' shape=(50257, 1024) dtype=float32, numpy=
array([[-0.03407753, -0.09780064,  0.19781208, ..., -0.3549903 ,
        -0.33180118,  0.04888699],
       [-0.24257368, -0.16181791,  0.00455345, ...,  0.03153262,
        -0.33765626,  0.03721819],
       [ 0.03019633, -0.04952642,  0.00953966, ...,  0.23675954,
        -0.3426564 ,  0.0298616 ],
       ...,
       [-0.21081589, -0.28816822,  0.00261875, ...,  0.00942319,
        -0.00367257,  0.02044453],
       [-0.31295916, -0.09149817, -0.12980784, ...,  0.44957328,
         0.03552138, -0.54266703],
       [ 0.11472895,  0.04638724,  0.16703497, ...,  0.06346995,
         0.11706788,  0.07779448]], dtype=float32)>

In [ ]:
# we are passing a dummy loss; by passing the labels argument we are already using the predefined lm loss function 
def dummy_loss(y_true, y_pred):
            return tf.reduce_mean(y_pred)

In [ ]:
optimizer = AdamWeightDecay()
model.compile(optimizer=optimizer, loss={"loss": dummy_loss})

In [ ]:
def sample_generator(dataset, tokenizer):
    # permutating the dataset; the state of sample ordering remains the same for each yield call
    sample_ordering = np.random.permutation(len(dataset))
    for sample_idx in sample_ordering:
        example = dataset[int(sample_idx)]
        #convert dictionaries to tensors
        example = {key: tf.convert_to_tensor(arr, dtype_hint=tf.int64) for key, arr in example.items()}
        yield example, example["labels"]  # TF needs some kind of labels, even if we don't use them
    return

In [ ]:
#partial creates a generator with train_dataset and tokenizer as set input
train_generator = partial(sample_generator, train_dataset, tokenizer)

#features are attention mask, input_ids and labels 
train_signature = {feature: tf.TensorSpec(shape=(None,), dtype=tf.int64) for feature in train_dataset.features}
train_sig = (train_signature, train_signature["labels"])

In [ ]:
eval_generator = partial(sample_generator, eval_dataset, tokenizer)
eval_signature = {feature: tf.TensorSpec(shape=(None,), dtype=tf.int64) for feature in eval_dataset.features }
eval_sig = (eval_signature, eval_signature["labels"])

In [ ]:
num_train_epochs = 1

In [ ]:
# tensorflow evokes the generator during the training process so that there is no excessive ram usage
# otherwise the model would run out of memory quickly
tf_eval_dataset = (
            tf.data.Dataset.from_generator(eval_generator, output_signature=eval_sig)
            .batch(batch_size=1, drop_remainder=True)
            .repeat(int(num_train_epochs))
)

In [ ]:
tf_train_dataset = (
            tf.data.Dataset.from_generator(train_generator, output_signature=train_sig)
            .batch(batch_size=1, drop_remainder=True)
            .repeat(int(num_train_epochs))
        )

In [ ]:
# the train_generator yields a dictionary of {attention_mask,input_ids, labels} and labels
next(tf_train_dataset.as_numpy_iterator())


({'attention_mask': array([[1, 1, 1, ..., 1, 1, 1]]),
  'input_ids': array([[ 198,  568,  611, ...,  470, 1394, 6225]]),
  'labels': array([[ 198,  568,  611, ...,  470, 1394, 6225]])},
 array([[ 198,  568,  611, ...,  470, 1394, 6225]]))

In [ ]:
class SavePretrainedCallback(tf.keras.callbacks.Callback):
    # Hugging Face models have a save_pretrained() method that saves both the weights and the necessary
    # metadata to allow them to be loaded as a pretrained model in future. This is a simple Keras callback
    # that saves the model with this method after each epoch.
    def __init__(self, output_dir, **kwargs):
        super().__init__()
        self.output_dir = output_dir

    def on_epoch_end(self, epoch, logs=None):
        self.model.save_pretrained(self.output_dir)

In [ ]:
output_dir = "/content/drive/MyDrive/NLP/GPT-2/Model"

In [ ]:
history = model.fit(
            tf_train_dataset,
            validation_data=tf_eval_dataset,
            epochs=num_train_epochs,
            steps_per_epoch=len(train_dataset), # len(train_dataset) = number of blocks (mostly containing 2-3 songs)
            callbacks=[SavePretrainedCallback(output_dir=output_dir)]
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
model.save_pretrained(output_dir)